# CNN-LSTM model for in-season crop type classification

This notebook implements a dual-branch recurrent and convolutional neural network for crop type classification. Output classes are corn, soybean, or other. The input to the model includes optical (Harmonized Landsat and Sentinel-2) and microwave (Sentinel-1) observations at 30m spatial resolution on days of the year (DOYs) selected based on growth stages detected in each pixel (greenup, peak growth, and senescence). Models are trained and tested at these three stages of the growing season as well. The labels for training and evaluation are from the USDA NASS Cropland Data Layer (CDL), which provides crop-specific land cover classes at 30m resolution.


## Import necessary libraries and packages

In [ ]:
# Standard libraries
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
from time import time

# ML libraries
import tensorflow as tf
import keras
from keras.layers.core import *
from keras.models import Sequential, Model, load_model
from keras.layers import Dense
from keras.layers import concatenate
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Conv1D, Conv2D
from keras.layers import MaxPool2D
from keras.layers import Dropout
from keras.utils import to_categorical
from keras.optimizers import Adam
from sklearn import metrics

# Define some constants
CORN = 0
SOYBEAN = 1
OTHER = 2
N_CLASSES = 3
N_BANDS = 6
N_TIMESTEPS = 3
BATCH_SIZE = 4096
N_EPOCHS = 25
HEIGHT = 3660
WIDTH = 3660

# Fix random seed for reproducibility
np.random.seed(42)

## Load the dataset
The data are stored as numpy arrays with dimension height x width x bands x timesteps. All of the reflectance values are in the range [0,1]. We also add two spectral indices (NDWI and LSWI) and SAR bands (VV, VH, incidence angle/IA).

In [ ]:
def add_lswi_channel(X):
    _X = np.ndarray([HEIGHT, WIDTH, X.shape[2]+1, N_TIMESTEPS])
    # copy the values from the original array
    for i in range(X.shape[2]):
        _X[:,:,i,:] = X[:,:,i,:]
    # calculate values for LSWI channel
    for i in range(N_TIMESTEPS):
        lswi = (X[:,:,NIR,i]-X[:,:,SWIR1,i])/(X[:,:,NIR,i]+X[:,:,SWIR1,i])
        _X[:,:,-1,i] = lswi
    # make sure we didn't introduce any NaNs
    _X[np.where(np.isnan(_X))] = 0
    return _X

def add_ndwi_channel(X):
    _X = np.ndarray([HEIGHT, WIDTH, X.shape[2]+1, N_TIMESTEPS])
    # copy the values from the original array
    for i in range(X.shape[2]):
        _X[:,:,i,:] = X[:,:,i,:]
    # calculate values for NDWI channel
    for i in range(N_TIMESTEPS):
        ndwi = (X[:,:,GREEN,i]-X[:,:,SWIR1,i])/(X[:,:,GREEN,i]+X[:,:,SWIR1,i])
        _X[:,:,-1,i] = ndwi
    # make sure we didn't introduce any NaNs
    _X[np.where(np.isnan(_X))] = 0
    return _X

def add_sar_channel(X, band, path):
    _X = np.ndarray([HEIGHT, WIDTH, X.shape[2]+1, N_TIMESTEPS])
    # copy the values from the original array
    for i in range(X.shape[2]):
        _X[:,:,i,:] = X[:,:,i,:]
    # load the corresponding SAR band
    if band=='vv' or band=='VV':
        sarpath = path.replace('pheno_timeseries', 'vv_timeseries')
        #sarpath = path.replace('fixed_timeseries', 'vv_timeseries')
    elif band=='vh' or band=='VH':
        sarpath = path.replace('pheno_timeseries', 'vh_timeseries')
        #sarpath = path.replace('fixed_timeseries', 'vh_timeseries')
    elif band=='ia' or band=='IA':
        sarpath = path.replace('pheno_timeseries', 'ia_timeseries')
        #sarpath = path.replace('fixed_timeseries', 'ia_timeseries')
    sar = np.load(sarpath).astype(np.float32)
    for i in range(N_TIMESTEPS):
        _X[:,:,-1,i] = sar[...,i]
    # make sure we didn't introduce any NaNs
    _X[np.where(np.isnan(_X))] = 0
    return _X

def load_data(x_path, y_path, flatten=True, convert_nans=True):
    # Load the time series image data
    X = np.load(x_path).astype(np.float32)
    # Load the associated labels
    Y = np.load(y_path).astype(np.int8)
    
    # Convert all the NaNs to zeros
    if convert_nans:
        X[np.where(np.isnan(X))] = 0
        
    X[np.where(X==0)] = 0.00000001
    # Add band indices
    X = add_lswi_channel(X)
    X = add_ndwi_channel(X)
    X = add_sar_channel(X, 'vv', x_path)
    X = add_sar_channel(X, 'vh', x_path)
    X = add_sar_channel(X, 'ia', x_path)
    if flatten:
        # Reduce the h x w x b x t dataset to h*w x b x t
        X = np.reshape(X, (X.shape[0]*X.shape[1], X.shape[2], X.shape[3]))
        Y = np.reshape(Y, (Y.shape[0]*Y.shape[1]))
    assert X.shape[0] == Y.shape[0] 
    return X, Y

The training data used was collected over MGRS tile 16TBL in northern Illinois in 2018 and 2017. For validation, we will use the pixels in the southwest quadrant of the 2017-2018 16TBL data. We load 2017 and 2018 separately first and then split them up geographically.

In [ ]:
X_train, y_train = load_data(x_path='data/inputs_3doy/pheno_timeseries_16TBL_2018.npy', y_path='data/cdl_labels_16TBL_2018.npy', flatten=False)

In [ ]:
X_val, y_val = load_data(x_path='data/inputs_3doy/pheno_timeseries_16TBL_2017.npy', y_path='data/cdl_labels_16TBL_2017.npy', flatten=False)

In [ ]:
dois_2018 = np.load('data/phenology/16TBL_2018_phenology.npy').astype(np.int16)
dois_2017 = np.load('data/phenology/16TBL_2017_phenology.npy').astype(np.int16)

The variable delta_s is the number of days elapsed between the senescence and greenup DOYs in each pixel. Similarly, delta_p is the number of days elapsed between the peak and greenup DOYs in each pixel.

In [ ]:
delta_2018_s = dois_2018[...,2]-dois_2018[...,0]
delta_2017_s = dois_2017[...,2]-dois_2017[...,0]

In [ ]:
delta_2018_p = dois_2018[...,1]-dois_2018[...,0]
delta_2017_p = dois_2017[...,1]-dois_2017[...,0]

The homogeneity mask is used to select training pixels only within regions of the CDL that are spatially consistent and thus more likely to be correctly classified in the CDL reference labels.

In [ ]:
mask_2018 = np.load('data/cdl_labels_16TBL_2018_homogmask_3x3.npy')
mask_2017 = np.load('data/cdl_labels_16TBL_2017_homogmask_3x3.npy')

We split the eastern half and northwest quadrant of the 2017 and 2018 data out for training.

In [ ]:
# use eastern half and NW quadrant of 2017 and 2018 for training
east_2018 = X_train[:,int(WIDTH/2):]
east_2018_y = y_train[:,int(WIDTH/2):]
east_2018_mask = mask_2018[:,int(WIDTH/2):]
east_2018_delta_p = delta_2018_p[:,int(WIDTH/2):]
east_2018_delta_s = delta_2018_s[:,int(WIDTH/2):]

east_2017 = X_val[:,int(WIDTH/2):]
east_2017_y = y_val[:,int(WIDTH/2):]
east_2017_mask = mask_2017[:,int(WIDTH/2):]
east_2017_delta_s = delta_2017_s[:,int(WIDTH/2):]
east_2017_delta_p = delta_2017_p[:,int(WIDTH/2):]

nw_2018 = X_train[:int(HEIGHT/2),:int(WIDTH/2)]
nw_2018_y = y_train[:int(HEIGHT/2),:int(WIDTH/2)]
nw_2018_mask = mask_2018[:int(HEIGHT/2),:int(WIDTH/2)]
nw_2018_delta_s = delta_2018_s[:int(HEIGHT/2),:int(WIDTH/2)]
nw_2018_delta_p = delta_2018_p[:int(HEIGHT/2),:int(WIDTH/2)]

nw_2017 = X_val[:int(HEIGHT/2),:int(WIDTH/2)]
nw_2017_y = y_val[:int(HEIGHT/2),:int(WIDTH/2)]
nw_2017_mask = mask_2017[:int(HEIGHT/2),:int(WIDTH/2)]
nw_2017_delta_s = delta_2017_s[:int(HEIGHT/2),:int(WIDTH/2)]
nw_2017_delta_p = delta_2017_p[:int(HEIGHT/2),:int(WIDTH/2)]

We split the southwestern quadrant of the 2017 and 2018 data out for validation.

In [ ]:
# Use SW quadrant of 2017 and 2018 for validation
sw_2018 = X_train[int(HEIGHT/2):,:int(WIDTH/2)]
sw_2018_y = y_train[int(HEIGHT/2):,:int(WIDTH/2)]
sw_2018_mask = mask_2018[int(HEIGHT/2):,:int(WIDTH/2)]
sw_2018_delta_s = delta_2018_s[int(HEIGHT/2):,:int(WIDTH/2)]
sw_2018_delta_p = delta_2018_p[int(HEIGHT/2):,:int(WIDTH/2)]

sw_2017 = X_val[int(HEIGHT/2):,:int(WIDTH/2)]
sw_2017_y = y_val[int(HEIGHT/2):,:int(WIDTH/2)]
sw_2017_mask = mask_2017[int(HEIGHT/2):,:int(WIDTH/2)]
sw_2017_delta_s = delta_2017_s[int(HEIGHT/2):,:int(WIDTH/2)]
sw_2017_delta_p = delta_2017_p[int(HEIGHT/2):,:int(WIDTH/2)]

The actual input to the model is a pair containing the spectrum at 3 DOYs in one pixel and a 5x5 patch surrounding that pixel. This function creates these input pairs from the larger rasters. There is an option to select a random subset or to get the pairs for all pixels in the raster.

In [ ]:
# Make pixel and patch pairs from the raster
def make_inputs(raster, labels, deltas_s, deltas_p, subset=None, kernel_size=5, mask=None):
    # Iterate through pixels with buffer for kernel size
    w = int(kernel_size/2)
    margin = w+kernel_size%2
    if subset:
        # Random rows and cols gives subset**2 examples
        rand_i = np.random.randint(0, raster.shape[0]-kernel_size, size=subset)
        rand_j = np.random.randint(0, raster.shape[1]-kernel_size, size=subset)
        # Initialize arrays to hold the examples
        pixels = []
        patches = []
        y = []
        delta_s = []
        delta_p = []
        for idx_i, i in enumerate(rand_i):
            for idx_j, j in enumerate(rand_j):
                # only take examples from homogeneous regions of CDL
                if np.any(mask[i-w:i+w+1, j-w:j+w+1] != 1):
                    continue
                # Store the pixel representation
                pixel = raster[i,j]
                # Store the patch representation
                patch = raster[i-w:i+w+1, j-w:j+w+1]
                if patch.shape[0] != 5 or patch.shape[1] != 5:
                    continue
                patch = np.reshape(patch, [patch.shape[0], patch.shape[1], patch.shape[2]*patch.shape[3]],order='F')
                patches.append(patch)
                pixels.append(pixel)
                # Store the label
                y.append(labels[i,j])
                # Store the senescence-greenup deltas
                delta_s.append(deltas_s[i,j])
                delta_p.append(deltas_p[i,j])
    else:
        # Initialize arrays to hold the examples
        pixels = np.ndarray([HEIGHT*WIDTH, N_BANDS, N_TIMESTEPS])
        patches = np.ndarray([HEIGHT*WIDTH, kernel_size, kernel_size, N_BANDS*N_TIMESTEPS])
        y = np.ndarray([HEIGHT*WIDTH]).astype(np.int8)
        delta_s = np.ndarray([HEIGHT*WIDTH]).astype(np.int32)
        delta_p = np.ndarray([HEIGHT*WIDTH]).astype(np.int32)
        for i in range(margin, raster.shape[0]-margin):
            for j in range(margin, raster.shape[1]-margin):
                # Store the pixel representation
                pixel = raster[i,j]
                pixels[i*WIDTH+j] = pixel
                # Store the patch representation
                patch = raster[i-w:i+w+1, j-w:j+w+1]
                patch = np.reshape(patch, [patch.shape[0], patch.shape[1], patch.shape[2]*patch.shape[3]], order='F')
                patches[i*WIDTH+j] = patch
                # Store the label
                y[i*WIDTH+j] = labels[i,j]   
                # Store the senescence-greenup deltas
                delta_s[i*WIDTH+j] = deltas_s[i,j]
                delta_p[i*WIDTH+j] = deltas_p[i,j]
                
    return np.array(pixels), np.array(patches), np.array(y), np.array(delta_s), np.array(delta_p)

Make the input pairs for each of the training regions. Note some need to be run multiple times if a pixel in the random subset falls on the edge of the raster.

In [ ]:
east_2018_px, east_2018_pa, east_2018_labels, east_2018_delta_s, east_2018_delta_p = make_inputs(east_2018, east_2018_y, east_2018_delta_s, east_2018_delta_p, subset=400, mask=east_2018_mask) # 2 times

In [ ]:
east_2017_px, east_2017_pa, east_2017_labels, east_2017_delta_s, east_2017_delta_p = make_inputs(east_2017, east_2017_y, east_2017_delta_s, east_2017_delta_p, subset=400, mask=east_2017_mask) # 4 times

In [ ]:
nw_2018_px, nw_2018_pa, nw_2018_labels, nw_2018_delta_s, nw_2018_delta_p = make_inputs(nw_2018, nw_2018_y, nw_2018_delta_s, nw_2018_delta_p, subset=100, mask=nw_2018_mask)

In [ ]:
nw_2017_px, nw_2017_pa, nw_2017_labels, nw_2017_delta_s, nw_2017_delta_p = make_inputs(nw_2017, nw_2017_y, nw_2017_delta_s, nw_2017_delta_p, subset=100, mask=nw_2017_mask)

Concatenate the eastern and northwestern regions to form the training examples.

In [ ]:
X_train_px = np.concatenate([east_2018_px, east_2017_px, nw_2018_px, nw_2017_px], axis=0)
X_train_pa = np.concatenate([east_2018_pa, east_2017_pa, nw_2018_pa, nw_2017_pa], axis=0)

In [ ]:
y_train = np.concatenate([east_2018_labels, east_2017_labels, nw_2018_labels, nw_2017_labels], axis=0)

In [ ]:
delta_train_s = np.concatenate([east_2018_delta_s, east_2017_delta_s, nw_2018_delta_s, nw_2017_delta_s], axis=0)

In [ ]:
delta_train_p = np.concatenate([east_2018_delta_s, east_2017_delta_s, nw_2018_delta_s, nw_2017_delta_p], axis=0)

Subsample the dataset to equalize the number of examples we have for each class.

In [ ]:
def subsample(X_px, X_pa, y, delta_s, delta_p):
    # Get the minimum number of samples in a class
    n_corn = len(np.where(y==CORN)[0])
    n_soy = len(np.where(y==SOYBEAN)[0])
    n_other = len(np.where(y==OTHER)[0])
    lim = np.min([n_corn, n_soy, n_other])
    # Sub-sample the classes to the same number
    X_px = np.concatenate([X_px[np.where(y==CORN)][:lim], X_px[np.where(y==SOYBEAN)][:lim], X_px[np.where(y==OTHER)][:lim]])
    X_pa = np.concatenate([X_pa[np.where(y==CORN)][:lim], X_pa[np.where(y==SOYBEAN)][:lim], X_pa[np.where(y==OTHER)][:lim]])
    delta_s = np.concatenate([delta_s[np.where(y==CORN)][:lim], delta_s[np.where(y==SOYBEAN)][:lim], delta_s[np.where(y==OTHER)][:lim]])
    delta_p = np.concatenate([delta_p[np.where(y==CORN)][:lim], delta_p[np.where(y==SOYBEAN)][:lim], delta_p[np.where(y==OTHER)][:lim]])
    y = np.concatenate([y[np.where(y==CORN)][:lim], y[np.where(y==SOYBEAN)][:lim], y[np.where(y==OTHER)][:lim]])
    return X_px, X_pa, y, delta_s, delta_p

In [ ]:
X_train_px, X_train_pa, y_train, delta_train_s, delta_train_p = subsample(X_train_px, X_train_pa, y_train, delta_train_s, delta_train_p)

Make the input pairs for each of the validation regions.

In [ ]:
sw_2018_px, sw_2018_pa, sw_2018_labels, sw_2018_delta_s, sw_2018_delta_p = make_inputs(sw_2018, sw_2018_y, sw_2018_delta_s, sw_2018_delta_p, subset=100, mask=sw_2018_mask)

In [ ]:
sw_2017_px, sw_2017_pa, sw_2017_labels, sw_2017_delta_s, sw_2017_delta_p = make_inputs(sw_2017, sw_2017_y, sw_2017_delta_s, sw_2017_delta_p, subset=100, mask=sw_2017_mask)

Concatenate the southwestern regions to form the validation examples.

In [ ]:
X_val_px = np.concatenate([sw_2018_px, sw_2017_px], axis=0)
X_val_pa = np.concatenate([sw_2018_pa, sw_2017_pa], axis=0)

In [ ]:
y_val = np.concatenate([sw_2018_labels, sw_2017_labels], axis=0)

In [ ]:
delta_val_s = np.concatenate([sw_2018_delta_s, sw_2017_delta_s], axis=0)

In [ ]:
delta_val_p = np.concatenate([sw_2018_delta_p, sw_2017_delta_p], axis=0)

Subsample the dataset to equalize the number of examples we have for each class.

In [ ]:
X_val_px, X_val_pa, y_val, delta_val_s, delta_val_p = subsample(X_val_px, X_val_pa, y_val, delta_val_s, delta_val_p)

### Swap temporal and spectral axes
Currently our data have dimension # pixels x # bands (features) x # time step. The LSTM expects them in the order # pixels x # time steps x # bands (features) so we need to re-order the last two dimensions.

In [ ]:
X_train_px = np.swapaxes(X_train_px, 1, 2)
X_val_px = np.swapaxes(X_val_px, 1, 2)

### Make one-hot labels
The labels for our three crop type classes---corn, soybean, and other---are currently integers. We need to make these one-hot vectors (e.g., corn: [1, 0, 0]).

In [ ]:
y_val = to_categorical(y_val, num_classes=N_CLASSES)

In [ ]:
y_train = to_categorical(y_train, num_classes=N_CLASSES)

Label smoothing has been shown to improve model generalization and learning speed: http://papers.nips.cc/paper/8717-when-does-label-smoothing-help

In [ ]:
# Smooth the labels
def smooth_labels(labels, factor=0.1):
    # smooth the labels
    labels = labels * (1 - factor)
    labels = labels + (factor / labels.shape[1])
    return labels

In [ ]:
y_train = smooth_labels(y_train)

In [ ]:
y_val = smooth_labels(y_val)

### Standardize the data
Standardize the data by subtracting the mean and dividing by the standard deviation

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Different scaler for each timestep
gscaler = StandardScaler()
pscaler = StandardScaler()
sscaler = StandardScaler()

In [ ]:
X_train_px[:,0] = gscaler.fit_transform(X_train_px[:,0])
X_train_px[:,1] = pscaler.fit_transform(X_train_px[:,1])
X_train_px[:,2] = sscaler.fit_transform(X_train_px[:,2])

In [ ]:
X_val_px[:,0] = gscaler.transform(X_val_px[:,0])
X_val_px[:,1] = pscaler.transform(X_val_px[:,1])
X_val_px[:,2] = sscaler.transform(X_val_px[:,2])

In [ ]:
mu_pa = np.mean(X_train_pa, axis=0)
std_pa = np.std(X_train_pa, axis=0)

In [ ]:
def standardize(X, mu, std):
    return (X - mu)/std

In [ ]:
X_train_pa = standardize(X_train_pa, mu_pa, std_pa)
X_val_pa = standardize(X_val_pa, mu_pa, std_pa)

Visualize an example of the standardised data

In [ ]:
t = 40
y_train[t]

In [ ]:
plt.plot(X_train_px[t,0],c='g',label='Greenup')
plt.plot(X_train_px[t,1],c='y',label='Peak')
plt.plot(X_train_px[t,2],c='r',label='Senescence')
plt.axis('off')
plt.legend()

In [ ]:
fig, axes = plt.subplots(nrows=N_TIMESTEPS, ncols=int(X_train_pa.shape[-1]/N_TIMESTEPS))
for b in range(int(X_train_pa.shape[-1]/N_TIMESTEPS)):
    for d in range(N_TIMESTEPS):
        axes[d,b].imshow(X_train_pa[t,:,:,d*int(X_train_pa.shape[-1]/N_TIMESTEPS) + b], cmap='gray')
        axes[d,b].axis('off')

## Train the model
We have separated out different components of the full model to evaluate the contribution of each component. Within each model or sub-model, there is a model for early-season, mid-season, and late-season classification.

In [ ]:
class PlotLearning(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('accuracy'))
        self.val_acc.append(logs.get('val_accuracy'))
        self.i += 1
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
        f.tight_layout()
        clear_output(wait=True)
        
        ax1.set_yscale('log')
        ax1.plot(self.x, self.losses, label="training loss")
        ax1.plot(self.x, self.val_losses, label="validation loss")
        ax1.legend()
        
        ax2.plot(self.x, self.acc, label="training accuracy")
        ax2.plot(self.x, self.val_acc, label="validation accuracy")
        ax2.legend()
        
        plt.show();
        
plot = PlotLearning()

## CNN only

### Early-season

In [ ]:
######## CNN branch only ########

cnn_inputs = Input(shape=(5, 5, 1*N_BANDS))

conv1_2d = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(cnn_inputs)

conv2_2d = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(conv1_2d)
mp1 = MaxPool2D()(conv2_2d)

flat_cnn = Flatten()(mp1)

fc = Dense(64, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(flat_cnn)

# output layer -> corn, soybeans, other crop, other non-crop
preds = Dense(N_CLASSES, activation='softmax')(fc)

# Build the model
model = Model(inputs=[cnn_inputs], outputs=preds)

# Compile the model with loss function and optimizer
adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.build(input_shape) 

# Train the model
model.fit(x=[X_train_pa[:,:,:,:1*N_BANDS]], y=y_train, validation_data=([X_val_pa[:,:,:,:1*N_BANDS]], y_val), epochs=N_EPOCHS, batch_size=BATCH_SIZE, callbacks=[plot])


### Mid-season

In [ ]:
######## CNN branch only ########

cnn_inputs = Input(shape=(5, 5, 2*N_BANDS))

conv1_2d = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(cnn_inputs)

conv2_2d = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(conv1_2d)
mp1 = MaxPool2D()(conv2_2d)

flat_cnn = Flatten()(mp1)

fc = Dense(64, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(flat_cnn)

# output layer -> corn, soybeans, other crop, other non-crop
preds = Dense(N_CLASSES, activation='softmax')(fc)

# Build the model
model = Model(inputs=[cnn_inputs], outputs=preds)

# Compile the model with loss function and optimizer
adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.build(input_shape) 

# Train the model
model.fit(x=[X_train_pa[:,:,:,:2*N_BANDS]], y=y_train, validation_data=([X_val_pa[:,:,:,:2*N_BANDS]], y_val), epochs=N_EPOCHS, batch_size=BATCH_SIZE, callbacks=[plot])


### Late-season

In [ ]:
######## CNN branch only ########

cnn_inputs = Input(shape=(5, 5, N_TIMESTEPS*N_BANDS))

conv1_2d = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(cnn_inputs)

conv2_2d = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(conv1_2d)
mp1 = MaxPool2D()(conv2_2d)

flat_cnn = Flatten()(mp1)

fc = Dense(64, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(flat_cnn)

# output layer -> corn, soybeans, other crop, other non-crop
preds = Dense(N_CLASSES, activation='softmax')(fc)

# Build the model
model = Model(inputs=[cnn_inputs], outputs=preds)

# Compile the model with loss function and optimizer
adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.build(input_shape) 

# Train the model
model.fit(x=[X_train_pa], y=y_train, validation_data=([X_val_pa], y_val), epochs=N_EPOCHS, batch_size=BATCH_SIZE, callbacks=[plot])


## LSTM only

The LSTM-only model does not have early-season classification because it does not make sense to use an LSTM with only one timestep (we only use the greenup DOY for early-season classification).

### Mid-season

In [ ]:
######## LSTM only ########
from keras.layers import BatchNormalization

input_shape = (2, N_BANDS)
lstm_inputs = Input(shape=input_shape)

conv1 = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_inputs)

lstm1 = LSTM(16, input_shape=conv1.shape, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(conv1)
drop1 = Dropout(0.3)(lstm1)

lstm2 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(drop1)
drop2 = Dropout(0.3)(lstm2)

# Concatenate the output of LSTM1 and LSTM2
lstm_dense_1 = concatenate([drop1, drop2])

lstm3 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_dense_1)
drop3 = Dropout(0.3)(lstm3)

# Concatenate the output of LSTM1, LSTM2, and LSTM3
lstm_dense_2 = concatenate([drop1, drop2, drop3])

lstm4 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_dense_2)
drop4 = Dropout(0.3)(lstm4)

# Concatenate the output of LSTM1, LSTM2, LSTM3, and LSTM4
lstm_dense_3 = concatenate([drop1, drop2, drop3, drop4])

flat_lstm = Flatten()(lstm_dense_3)

#fully-connected layer
fc = Dense(64, activation='relu')(flat_lstm)

# output layer -> corn, soybeans, other crop, other non-crop
preds = Dense(N_CLASSES, activation='softmax')(fc)

# Build the model
model = Model(inputs=[lstm_inputs], outputs=preds)

# Compile the model with loss function and optimizer
adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.build(input_shape) 

# Train the model
model.fit(x=[X_train_px[:,:2,:]], y=y_train, validation_data=([X_val_px[:,:2,:]], y_val), epochs=N_EPOCHS, batch_size=BATCH_SIZE, callbacks=[plot])


### Late-season

In [ ]:
######## LSTM only ########
from keras.layers import BatchNormalization

input_shape = (N_TIMESTEPS, N_BANDS)
lstm_inputs = Input(shape=input_shape)

conv1 = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_inputs)

lstm1 = LSTM(16, input_shape=conv1.shape, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(conv1)
# bn2 = BatchNormalization()(lstm1)
drop1 = Dropout(0.3)(lstm1)

lstm2 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(drop1)
drop2 = Dropout(0.3)(lstm2)

# Concatenate the output of LSTM1 and LSTM2
lstm_dense_1 = concatenate([drop1, drop2])

lstm3 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_dense_1)
drop3 = Dropout(0.3)(lstm3)

# Concatenate the output of LSTM1, LSTM2, and LSTM3
lstm_dense_2 = concatenate([drop1, drop2, drop3])

lstm4 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_dense_2)
drop4 = Dropout(0.3)(lstm4)

# Concatenate the output of LSTM1, LSTM2, LSTM3, and LSTM4
lstm_dense_3 = concatenate([drop1, drop2, drop3, drop4])

flat_lstm = Flatten()(lstm_dense_3)

#fully-connected layer
fc = Dense(64, activation='relu')(flat_lstm)

# output layer -> corn, soybeans, other crop, other non-crop
preds = Dense(N_CLASSES, activation='softmax')(fc)

# Build the model
model = Model(inputs=[lstm_inputs], outputs=preds)

# Compile the model with loss function and optimizer
adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.build(input_shape) 

# Train the model
model.fit(x=[X_train_px], y=y_train, validation_data=([X_val_px], y_val), epochs=N_EPOCHS, batch_size=BATCH_SIZE, callbacks=[plot])

## CNN-LSTM-delta

Again, there is no early-season classification because of the LSTM component.

### Mid-season LSTM-CNN_d

In [ ]:
######## LSTM branch ########
from keras.layers import BatchNormalization

input_shape = (2, N_BANDS)
lstm_inputs = Input(shape=input_shape)

conv1 = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_inputs)
bn1 = BatchNormalization()(conv1)

lstm1 = LSTM(16, input_shape=conv1.shape, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(bn1)
bn2 = BatchNormalization()(lstm1)
drop1 = Dropout(0.3)(bn2)

lstm2 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(drop1)
bn3 = BatchNormalization()(lstm2)
drop2 = Dropout(0.3)(bn3)

# Concatenate the output of LSTM1 and LSTM2
lstm_dense_1 = concatenate([drop1, drop2])

lstm3 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_dense_1)
bn4 = BatchNormalization()(lstm3)
drop3 = Dropout(0.3)(bn4)

# Concatenate the output of LSTM1, LSTM2, and LSTM3
lstm_dense_2 = concatenate([drop1, drop2, drop3])

lstm4 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_dense_2)
bn5 = BatchNormalization()(lstm4)
drop4 = Dropout(0.3)(bn5)

# Concatenate the output of LSTM1, LSTM2, LSTM3, and LSTM4
lstm_dense_3 = concatenate([drop1, drop2, drop3, drop4])

flat_lstm = Flatten()(lstm_dense_3)

# Build the LSTM branch
lstm = Model(inputs=lstm_inputs, outputs=flat_lstm)

######## CNN branch ########

cnn_inputs = Input(shape=(5, 5, 2*N_BANDS))

conv1_2d = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(cnn_inputs)
bn6 = BatchNormalization()(conv1_2d)

conv2_2d = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(bn6)
bn7 = BatchNormalization()(conv2_2d)
mp1 = MaxPool2D()(bn7)

flat_cnn = Flatten()(mp1)

# Build the CNN branch
cnn = Model(inputs=cnn_inputs, outputs=flat_cnn)

########## Scalar timeline branch ###########
scalar_inputs = Input(shape=(1,))

combined = concatenate([lstm.output, cnn.output, scalar_inputs])

fc = Dense(64, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(combined)

# output layer -> corn, soybeans, other 
preds = Dense(N_CLASSES, activation='softmax')(fc)

# Build the model
model = Model(inputs=[lstm.input, cnn.input, scalar_inputs], outputs=preds)

# Compile the model with loss function and optimizer
adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.build(input_shape) 

# Train the model
model.fit(x=[X_train_px[:,:2], X_train_pa[...,:2*N_BANDS], delta_train_p], y=y_train, validation_data=([X_val_px[:,:2], X_val_pa[...,:2*N_BANDS], delta_val_p], y_val), epochs=N_EPOCHS, batch_size=BATCH_SIZE, callbacks=[plot])

### Mid-season LSTM-CNN (no delta)

In [ ]:
######## LSTM branch ########
from keras.layers import BatchNormalization

input_shape = (2, N_BANDS)
lstm_inputs = Input(shape=input_shape)

conv1 = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_inputs)
bn1 = BatchNormalization()(conv1)

lstm1 = LSTM(16, input_shape=conv1.shape, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(bn1)
bn2 = BatchNormalization()(lstm1)
drop1 = Dropout(0.3)(bn2)

lstm2 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(drop1)
bn3 = BatchNormalization()(lstm2)
drop2 = Dropout(0.3)(bn3)

# Concatenate the output of LSTM1 and LSTM2
lstm_dense_1 = concatenate([drop1, drop2])

lstm3 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_dense_1)
bn4 = BatchNormalization()(lstm3)
drop3 = Dropout(0.3)(bn4)

# Concatenate the output of LSTM1, LSTM2, and LSTM3
lstm_dense_2 = concatenate([drop1, drop2, drop3])

lstm4 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_dense_2)
bn5 = BatchNormalization()(lstm4)
drop4 = Dropout(0.3)(bn5)

# Concatenate the output of LSTM1, LSTM2, LSTM3, and LSTM4
lstm_dense_3 = concatenate([drop1, drop2, drop3, drop4])

flat_lstm = Flatten()(lstm_dense_3)

# Build the LSTM branch
lstm = Model(inputs=lstm_inputs, outputs=flat_lstm)

######## CNN branch ########

cnn_inputs = Input(shape=(5, 5, 2*N_BANDS))

conv1_2d = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(cnn_inputs)
bn6 = BatchNormalization()(conv1_2d)

conv2_2d = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(bn6)
bn7 = BatchNormalization()(conv2_2d)
mp1 = MaxPool2D()(bn7)

flat_cnn = Flatten()(mp1)

# Build the CNN branch
cnn = Model(inputs=cnn_inputs, outputs=flat_cnn)

combined = concatenate([lstm.output, cnn.output])

fc = Dense(64, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(combined)

# output layer -> corn, soybeans, other 
preds = Dense(N_CLASSES, activation='softmax')(fc)

# Build the model
model = Model(inputs=[lstm.input, cnn.input], outputs=preds)

# Compile the model with loss function and optimizer
adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.build(input_shape) 

# Train the model
model.fit(x=[X_train_px[:,:2], X_train_pa[...,:2*N_BANDS]], y=y_train, validation_data=([X_val_px[:,:2], X_val_pa[...,:2*N_BANDS]], y_val), epochs=N_EPOCHS, batch_size=BATCH_SIZE, callbacks=[plot])

### Late-season LSTM-CNN_d

In [ ]:
######## LSTM branch ########
from keras.layers import BatchNormalization
np.random.seed(42)

input_shape = (3, X_train_px.shape[-1])
lstm_inputs = Input(shape=input_shape)

conv1 = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_inputs)
bn1 = BatchNormalization()(conv1)

lstm1 = LSTM(16, input_shape=conv1.shape, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(bn1)
bn2 = BatchNormalization()(lstm1)
drop1 = Dropout(0.3)(bn2)

lstm2 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(drop1)
bn3 = BatchNormalization()(lstm2)
drop2 = Dropout(0.3)(bn3)

# Concatenate the output of LSTM1 and LSTM2
lstm_dense_1 = concatenate([drop1, drop2])

lstm3 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_dense_1)
bn4 = BatchNormalization()(lstm3)
drop3 = Dropout(0.3)(bn4)

# Concatenate the output of LSTM1, LSTM2, and LSTM3
lstm_dense_2 = concatenate([drop1, drop2, drop3])

lstm4 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_dense_2)
bn5 = BatchNormalization()(lstm4)
drop4 = Dropout(0.3)(bn5)

# Concatenate the output of LSTM1, LSTM2, LSTM3, and LSTM4
lstm_dense_3 = concatenate([drop1, drop2, drop3, drop4])

flat_lstm = Flatten()(lstm_dense_3)

# Build the LSTM branch
lstm = Model(inputs=lstm_inputs, outputs=flat_lstm)

######## CNN branch ########

cnn_inputs = Input(shape=(5, 5, 3*N_BANDS))

conv1_2d = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(cnn_inputs)
bn6 = BatchNormalization()(conv1_2d)

conv2_2d = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(bn6)
bn7 = BatchNormalization()(conv2_2d)
mp1 = MaxPool2D()(bn7)

flat_cnn = Flatten()(mp1)

# Build the CNN branch
cnn = Model(inputs=cnn_inputs, outputs=flat_cnn)

########## Scalar timeline branch ###########
scalar_inputs = Input(shape=(1,))

combined = concatenate([lstm.output, cnn.output, scalar_inputs])

fc = Dense(64, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(combined)

# output layer -> corn, soybeans, other 
preds = Dense(N_CLASSES, activation='softmax')(fc)

# Build the model
model = Model(inputs=[lstm.input, cnn.input, scalar_inputs], outputs=preds)
np.random.seed(42)
# Compile the model with loss function and optimizer
adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.build(input_shape) 

# Train the model
model.fit(x=[X_train_px, X_train_pa, delta_train_s], y=y_train, validation_data=([X_val_px, X_val_pa, delta_val_s], y_val), epochs=N_EPOCHS, batch_size=BATCH_SIZE, callbacks=[plot])

## Late-season LSTM-CNN (no delta)

In [ ]:
######## LSTM branch ########
from keras.layers import BatchNormalization

input_shape = (3, N_BANDS)
lstm_inputs = Input(shape=input_shape)

conv1 = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_inputs)
bn1 = BatchNormalization()(conv1)

lstm1 = LSTM(16, input_shape=conv1.shape, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(bn1)
bn2 = BatchNormalization()(lstm1)
drop1 = Dropout(0.3)(bn2)

lstm2 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(drop1)
bn3 = BatchNormalization()(lstm2)
drop2 = Dropout(0.3)(bn3)

# Concatenate the output of LSTM1 and LSTM2
lstm_dense_1 = concatenate([drop1, drop2])

lstm3 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_dense_1)
bn4 = BatchNormalization()(lstm3)
drop3 = Dropout(0.3)(bn4)

# Concatenate the output of LSTM1, LSTM2, and LSTM3
lstm_dense_2 = concatenate([drop1, drop2, drop3])

lstm4 = LSTM(16, return_sequences=True, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(lstm_dense_2)
bn5 = BatchNormalization()(lstm4)
drop4 = Dropout(0.3)(bn5)

# Concatenate the output of LSTM1, LSTM2, LSTM3, and LSTM4
lstm_dense_3 = concatenate([drop1, drop2, drop3, drop4])

flat_lstm = Flatten()(lstm_dense_3)

# Build the LSTM branch
lstm = Model(inputs=lstm_inputs, outputs=flat_lstm)

######## CNN branch ########

cnn_inputs = Input(shape=(5, 5, 3*N_BANDS))

conv1_2d = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(cnn_inputs)
bn6 = BatchNormalization()(conv1_2d)

conv2_2d = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(bn6)
bn7 = BatchNormalization()(conv2_2d)
mp1 = MaxPool2D()(bn7)

flat_cnn = Flatten()(mp1)

# Build the CNN branch
cnn = Model(inputs=cnn_inputs, outputs=flat_cnn)

combined = concatenate([lstm.output, cnn.output])

fc = Dense(64, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=4))(combined)

# output layer -> corn, soybeans, other 
preds = Dense(N_CLASSES, activation='softmax')(fc)

# Build the model
model = Model(inputs=[lstm.input, cnn.input], outputs=preds)
np.random.seed(42)
# Compile the model with loss function and optimizer
adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.build(input_shape) 

# Train the model
model.fit(x=[X_train_px, X_train_pa], y=y_train, validation_data=([X_val_px, X_val_pa], y_val), epochs=N_EPOCHS, batch_size=BATCH_SIZE, callbacks=[plot])

In [ ]:
model.save('results/models/cnn_lstm_train=16tbl201718_val=16tbl201718_epochs=%d_batchsize=%d' % (N_EPOCHS, BATCH_SIZE))

## 4. Test the model

In [ ]:
model = load_model('results/models/cnn_lstm_train=16tbl201718_val=16tbl201718_epochs=%d_batchsize=%d' % (N_EPOCHS, BATCH_SIZE))

In [ ]:
X_test, y_labels = load_data(x_path='data/inputs_3doy/pheno_timeseries_16TBL_2019.npy', y_path='data/cdl_labels_16TBL_2019.npy', flatten=False)

In [ ]:
dois_2019 = np.load('data/phenology/16TBL_2019_phenology.npy').astype(np.int16)

In [ ]:
delta_test_s = dois_2019[...,2]-dois_2019[...,0]

In [ ]:
delta_test_p = dois_2019[...,1]-dois_2019[...,0]

In [ ]:
X_test_px, X_test_pa, y_test, delta_test_s, delta_test_p = make_inputs(X_test, y_labels, delta_test_s, delta_test_p, subset=None)

In [ ]:
X_test_px = np.swapaxes(X_test_px, 1, 2)

In [ ]:
y_test = to_categorical(y_test, num_classes=N_CLASSES)

### Standardize the data

In [ ]:
X_test_px[:,0] = gscaler.transform(X_test_px[:,0])
X_test_px[:,1] = pscaler.transform(X_test_px[:,1])
X_test_px[:,2] = sscaler.transform(X_test_px[:,2])

In [ ]:
X_test_pa = standardize(X_test_pa, mu_pa, std_pa)

## CNN only

### Early-season

In [ ]:
pred = model.predict([X_test_pa[...,:1*N_BANDS]])

### Mid-season

In [ ]:
pred = model.predict([X_test_pa[...,:2*N_BANDS]])

### Late-season

In [ ]:
pred = model.predict([X_test_pa])

## LSTM only

### Mid-season

In [ ]:
pred = model.predict([X_test_px[:,:2]])

### Late-season

In [ ]:
pred = model.predict([X_test_px])

## CNN-LSTM

### Mid-season

In [ ]:
pred = model.predict([X_test_px[:,:2], X_test_pa[...,:2*N_BANDS]])

### Late-season 

In [ ]:
pred = model.predict([X_test_px, X_test_pa])

## CNN-LSTM-delta

### Mid-season LSTM-CNN_d

In [ ]:
pred = model.predict([X_test_px[:,:2], X_test_pa[...,:2*N_BANDS], delta_test_p])

### Late-season LSTM-CNN_d

In [ ]:
pred = model.predict([X_test_px, X_test_pa, delta_test_s])

### Compute accuracy metrics

In [ ]:
y_true_class = np.argmax(y_test, axis=1)

In [ ]:
y_true_class = np.reshape(y_true_class, [HEIGHT, WIDTH])

In [ ]:
y_pred_class = np.argmax(pred, axis=1)

In [ ]:
y_pred_class = np.reshape(y_pred_class, [HEIGHT, WIDTH])

Overall accuracy

In [ ]:
print("Test accuracy: %f" % metrics.accuracy_score(y_true_class[3:-3,3:-3].flatten(), y_pred_class[3:-3,3:-3].flatten()))

Confusion matrix (un-normalized)

In [ ]:
print(metrics.confusion_matrix(y_true_class[3:-3,3:-3].flatten(), y_pred_class[3:-3,3:-3].flatten()))

Normalized confusion matrix by row (recall/producers accuracy)

In [ ]:
def confusion_matrix_recall(y_true, y_pred):
    conf = metrics.confusion_matrix(y_true.flatten(), y_pred.flatten())
    _conf = np.zeros(conf.shape)
    for row in range(conf.shape[0]): # each row is for one class
        total = np.sum(conf[row])
        _conf[row] = conf[row]/float(total)
    return _conf

In [ ]:
print(confusion_matrix_recall(y_true_class[3:-3,3:-3].flatten(), y_pred_class[3:-3,3:-3].flatten()))

Normalized confusion matrix by column (precision/producer's accuracy)

In [ ]:
def confusion_matrix_precision(y_true, y_pred):
    conf = metrics.confusion_matrix(y_true.flatten(), y_pred.flatten())
    _conf = np.zeros(conf.shape)
    for col in range(conf.shape[1]): # each row is for one class
        total = np.sum(conf[:,col])
        _conf[:,col] = conf[:,col]/float(total)
    return _conf

In [ ]:
print(confusion_matrix_precision(y_true_class[3:-3,3:-3].flatten(), y_pred_class[3:-3,3:-3].flatten()))

Visualize the results compared to CDL labels

In [ ]:
def class_to_rgb(Y):
    Y_rgb = np.ndarray([Y.shape[0], Y.shape[1], 3])
    Y_rgb[np.where(Y==CORN)] = [1, 0.82, 0] # yellow
    Y_rgb[np.where(Y==OTHER)] = [0.8, 0.8, 0.8] # gray
    Y_rgb[np.where(Y==SOYBEAN)] = [0.149, 0.439, 0] # green
    return Y_rgb

In [ ]:
plt.imshow(class_to_rgb(y_true_class))
plt.title('CDL 2019')
plt.axis('off')

In [ ]:
plt.imshow(class_to_rgb(y_true_class[500:1000,500:1000]))
plt.title('CDL 2019')
plt.axis('off')

In [ ]:
plt.imshow(class_to_rgb(y_pred_class[500:1000,500:1000]))
plt.title('Predicted 2019')
plt.axis('off')

In [ ]:
plt.imshow(class_to_rgb(y_pred_class))
plt.title('Predicted 2019')
plt.axis('off')